In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/ddr/ddr.zip'

In [1]:
import keras
from keras import backend as K
import numpy as np
import glob
import shutil
from tensorflow.keras import regularizers
from keras import layers
from sklearn.model_selection import StratifiedKFold
from keras.layers import GlobalAveragePooling2D,Conv2D,Multiply,BatchNormalization,AveragePooling2D,GlobalMaxPool2D
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.densenet import DenseNet121
from keras.applications.inception_v3 import InceptionV3 
from keras.applications.densenet import DenseNet121
from keras.applications.vgg19 import VGG19
from keras.applications import ResNet50
from keras.layers.core import Lambda
from sklearn.metrics import roc_auc_score
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.layers import Dense,Reshape,Activation,Permute,Dot,Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
# from keras.utils import multi_gpu_model
from matplotlib import pyplot as plt
from keras.models import load_model
import os

In [2]:
def Global_Attention_Block(inputs):
        shape=K.int_shape(inputs)
#         avg_pool=GlobalAveragePooling2D()(inputs)
        avg_pool=AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
        avg_pool=Conv2D(shape[3],1,padding='same')(avg_pool)
        avg_pool=Activation('sigmoid')(avg_pool)
        avg_pool=Conv2D(shape[3],1,padding='same')(avg_pool)
        avg_pool=Activation('sigmoid')(avg_pool)
        
        C_A= Multiply()([inputs,avg_pool])
        avg_pool=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))(C_A)
        avg_pool=Activation('sigmoid')(avg_pool)
        S_A= Multiply()([avg_pool,C_A])
        return S_A

In [3]:
def Category_Attention_Block(inputs,classes,k):
    shape=K.int_shape(inputs)
    F_1=Conv2D(k*classes,1,padding='same')(inputs)
    F_1=BatchNormalization()(F_1)
    F1=Activation('sigmoid')(F_1)
    
    F_2=F1
    x=GlobalMaxPool2D()(F_2)
    x=Reshape((classes,k)) (x)
    S=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))(x)
    
    x=Reshape((shape[1],shape[2],classes,k)) (F1)
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))(x)
    x=Multiply()([S,x])
    M=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))(x)
    
    semantic=Multiply()([inputs,M])
    return semantic

In [4]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points    

In [5]:
   
def plotmodel(history,name):
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1) 
    
    plt.figure(1)                  
    plt.plot(epochs,smooth_curve(acc))
    plt.plot(epochs,smooth_curve(val_acc))
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_acc', 'val_accuracy'], loc='upper left')
    plt.title(name)
    plt.savefig('acc_'+name+'.png')
    
    plt.figure(2)
    plt.plot(epochs,smooth_curve(loss))
    plt.plot(epochs,smooth_curve(val_loss))
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'val_loss'], loc='upper right')
    plt.title(name)
    plt.savefig('loss_'+name+'.png')

In [6]:
def get_base_model(model_name,img_size):
    if(model_name=='vgg19'):
        base_model=VGG19(include_top=False,weights='imagenet',input_shape=(img_size,img_size,3))
    if model_name =='densenet121':
        base_model=DenseNet121       (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if(model_name=='inceptionv3'):
        base_model=InceptionV3(include_top=False,weights='imagenet',input_shape=(img_size,img_size,3))
    if(model_name=='resnet50'):
        base_model=ResNet50(include_top=False,weights='imagenet',input_shape=(img_size,img_size,3))
    if(model_name=='mobilenet'):
        base_model=MobileNet(include_top=False,weights='imagenet',input_shape=(img_size,img_size,3))
    if(model_name=='mobilenet1.0'):
        base_model=MobileNet(include_top=False,weights='imagenet',alpha=1.0,input_shape=(img_size,img_size,3))
    if(model_name=='xception'):
        base_model=Xception(include_top=False,weights='imagenet',input_shape=(img_size,img_size,3))
    return base_model

In [7]:
# def SelfAttention(inputs,transformer_layers):
#   shape=K.int_shape(inputs)
#   num_filters=shape[-1]

#   for _ in range(transformer_layers):
#   #query
#    x_q=Conv2D(num_filters,(1,1),padding='same')(inputs)
#    x_q=Activation('relu')(x_q)
#    x_qnew=Reshape((shape[1]*shape[2],shape[3]))(x_q)

#   #key
#    x_k=Conv2D(num_filters,(1,1),padding='same')(inputs)
#    x_k=Activation('relu')(x_k)
#    x_knew=Reshape((shape[3],shape[1]*shape[2]))(x_k)

#   #value
#    x_v=Conv2D(num_filters,(1,1),padding='same')(inputs)
#    x_v=Activation('relu')(x_v)
#    x_vnew=Reshape((shape[1]*shape[2],shape[3]))(x_v)


#    x=Dot(axes=(2, 1))([x_qnew, x_knew])


#    x1=Activation('softmax')(x)

#    x2=Dot(axes=(2, 1))([x1, x_vnew])

#    x_final=Reshape((1,shape[1]*shape[2],shape[3]))(x2)
#    x_final=Conv2D(num_filters,(1,1),padding='same')(x_final)
  
#    inputs=Reshape((shape[1],shape[2],shape[3]))(x_final)
#   return inputs




In [8]:
def SelfAttention(inputs):
   shape=K.int_shape(inputs)
   num_filters=shape[-1]
  #query
   x_q=Conv2D(num_filters,(1,1),padding='same')(inputs)
   x_q=Activation('relu')(x_q)
   x_qnew=Reshape((shape[1]*shape[2],shape[3]))(x_q)

  #key
   x_k=Conv2D(num_filters,(1,1),padding='same')(inputs)
   x_k=Activation('relu')(x_k)
   x_knew=Reshape((shape[3],shape[1]*shape[2]))(x_k)

  #value
   x_v=Conv2D(num_filters,(1,1),padding='same')(inputs)
   x_v=Activation('relu')(x_v)
   x_vnew=Reshape((shape[1]*shape[2],shape[3]))(x_v)


   x=Dot(axes=(2, 1))([x_qnew, x_knew])


   x1=Activation('softmax')(x)

   x2=Dot(axes=(2, 1))([x1, x_vnew])

   x_final=Reshape((1,shape[1]*shape[2],shape[3]))(x2)
   x_final=Conv2D(num_filters,(1,1),padding='same')(x_final)
  
   x_final=Reshape((shape[1],shape[2],shape[3]))(x_final)
  #  print(x_final.shape)
   return x_final




In [13]:
def train_model(model,image_size,batch_size,save_name,lr1,lr2,Epochs1,Epochs2):
#     dataParam={'messidor': [667,242,2,'./dataset/train','./dataset/test']}
    
#     train_num,valid_num,classes,train_dir,test_dir = dataParam[dataset]


  #  num_folds = 5


#  image_files = glob.glob(train_dir + '/**/*.tif', recursive=True)


#  labels = [img_file.split('/')[-2] for img_file in image_files]


#  class_mapping = {'non-referrable': 0, 'referrable': 1}
#  numeric_labels = [class_mapping[label] for label in labels]

  
#  classes=['non-referrable','referrable']


#  X = np.array(image_files)
#  y = np.array(numeric_labels)
#  skf = StratifiedKFold(n_splits=num_folds, shuffle=True)

#  for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
#     print('Fold:', fold+1)
#     print('Training samples:', len(train_index))
#     print('Validation samples:', len(val_index))
#     train_dir_fold = 'train_fold_{}'.format(fold+1)
#     val_dir_fold = 'val_fold_{}'.format(fold+1)
    
#     os.makedirs(train_dir_fold, exist_ok=True)
#     os.makedirs(val_dir_fold, exist_ok=True)
    
#     X_train, X_val = X[train_index], X[val_index]
#     y_train, y_val = y[train_index], y[val_index]
    
#     for class_label in classes:
#         os.makedirs(os.path.join(train_dir_fold, class_label), exist_ok=True)
    
#     for class_label in classes:
#         os.makedirs(os.path.join(val_dir_fold, class_label), exist_ok=True)
    
#     for i in range(len(X_train)):
#         class_label = labels[train_index[i]]
#         shutil.copy2(X_train[i], os.path.join(train_dir_fold, class_label, X_train[i].split('/')[-1]))
        

#     for i in range(len(X_val)):
#         class_label = labels[val_index[i]]
#         shutil.copy2(X_val[i], os.path.join(val_dir_fold, class_label, X_val[i].split('/')[-1]))
    train_dir_fold='ddr/train/'
    val_dir_fold='ddr/validation/'
    image_size=224
    batch_size=16
    # train_dir = '/content/drive/MyDrive/Messidor/data'
    train_num,valid_num=6260,2503 

    train=ImageDataGenerator(rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2)     

    valid = ImageDataGenerator(rescale=1./255)
      
    train_data = train.flow_from_directory(train_dir_fold, target_size=(image_size, image_size),shuffle=True,
                                                  batch_size=batch_size)
    val_data = valid.flow_from_directory(val_dir_fold, target_size=(image_size, image_size),shuffle=False,
                                                 batch_size=batch_size)
                                                  
      
    
    
    
    lr_decay=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=2)
    save_model=ModelCheckpoint('new/'+save_name+'{epoch:02d}.h5', monitor='val_loss',save_best_only=True,mode='min')
    
    for layer in base_model.layers:
         layer.trainable=False

    model.compile(optimizer=Adam(learning_rate=lr1,decay=0.00001),loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(train_data,steps_per_epoch=train_num/batch_size,validation_data=val_data,
                        validation_steps=valid_num/batch_size,epochs=Epochs1,workers=2,
                       callbacks=[lr_decay,save_model])
     
    for layer in base_model.layers:
         layer.trainable = True
        
    model.compile(optimizer=Adam(learning_rate=lr2,decay=0.00001),loss='categorical_crossentropy',metrics=['accuracy'])
    history=model.fit(train_data,steps_per_epoch=train_num/batch_size
                        ,validation_data=val_data,validation_steps=valid_num/batch_size,epochs=Epochs2,workers=2,
                       callbacks=[lr_decay,save_model])

    return history

In [14]:
k=5
lr1=0.005
lr2=0.0001
batch_size= 16
image_size=224
classes=5


base_model=get_base_model('mobilenet1.0',image_size)
base_in=base_model.input
base_out=base_model.output
num_filters=int(base_out.shape[-1])
# compress=Conv2D(num_filters/2,(1,1),padding='same')(base_out)

# x=Global_Attention_Block(compress)
# compress.shape[2]
x=SelfAttention(base_out)
# x=Category_Attention_Block(x,classes,k)

x=GlobalAveragePooling2D()(x)
x=Dropout(0.2)(x)
out=Dense(classes,activation='softmax',kernel_regularizer=regularizers.l2(0.001))(x)

parallel_model=Model(base_model.input,out)
parallel_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 112, 112, 32  128         ['conv1[0][0]']                  
                                )                                                           

In [15]:
history=train_model(parallel_model,image_size,batch_size,'mobilenet1.0',lr1,lr2,1,20)


Found 6260 images belonging to 5 classes.
Found 2503 images belonging to 5 classes.
391/391 [==============================] - 356s 906ms/step - loss: 1.4288 - accuracy: 0.4890 - val_loss: 1.1188 - val_accuracy: 0.5006 - lr: 0.0050
Epoch 1/20
  8/391 [..............................] - ETA: 18:11 - loss: 1.3003 - accuracy: 0.4375

In [ ]:
plotmodel(history,'resnet50')

In [ ]:
# print(auc)

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model

def get_gradcam(image, model):
    grad_model = Model(inputs=model.input, outputs=model.get_layer('Global_Attention_Block').output)

    with tf.GradientTape() as tape:
        conv_outputs = grad_model(image)

    grads = tape.gradient(conv_outputs, model.output)
    gate_f = tf.cast(conv_outputs > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = tf.cast(conv_outputs > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

    weights = tf.reduce_mean(guided_grads, axis=(0, 1, 2))
    cam = np.ones(conv_outputs.shape[1:3], dtype=np.float32)

    for i, w in enumerate(weights):
        cam += w * conv_outputs[0, :, :, i]

    cam = cv2.resize(cam.numpy(), (image.shape[2], image.shape[1]))
    cam = np.maximum(cam, 0)
    heatmap = (cam - cam.min()) / (cam.max() - cam.min())

    return heatmap

def generate_gradcam_GAB(image, model):
    heatmap = get_gradcam(image, model)

    # Apply GradCAM visualization on the original image
    heatmap = cv2.resize(heatmap, (image.shape[2], image.shape[1]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    gradcam = (heatmap * 0.5) + (image[0] * 0.5)

    return gradcam
